Data cleaning of the "Framingham" dataset, consisting of 15 health attributes plus ten year CHD outcome. A total of 4238 entries are present in this dataset. 

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('framingham.csv')
df.head(10)

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0
5,0,43,2.0,0,0.0,0.0,0,1,0,228.0,180.0,110.0,30.30,77.0,99.0,0
6,0,63,1.0,0,0.0,0.0,0,0,0,205.0,138.0,71.0,33.11,60.0,85.0,1
7,0,45,2.0,1,20.0,0.0,0,0,0,313.0,100.0,71.0,21.68,79.0,78.0,0
8,1,52,1.0,0,0.0,0.0,0,1,0,260.0,141.5,89.0,26.36,76.0,79.0,0
9,1,43,1.0,1,30.0,0.0,0,1,0,225.0,162.0,107.0,23.61,93.0,88.0,0


<h4><center>Table 1. Dataset Attribute Descriptions</center></h4>

| **Col. No.** 	|  **Attribute**  	|                            **Description**                            	|
|:------------:	|:---------------:	|:---------------------------------------------------------------------:	|
|       1      	|       male      	|                          1 = male, 0 = female                         	|
|       2      	|       age       	|                          Numeric age in years                         	|
|       3      	| education       	| Level of education 1-4, not specified                                 	|
|       4      	|  currentSmoker  	|              1 = current smoker, 0 = not a current smoker             	|
|       5      	|    cigsPerDay   	|             Number of cigarettes smoked per day on average            	|
|       6      	|      BPMeds     	|    1 = using blood pressure meds, 0 = not using BP meds   	|
|       7      	| prevalentStroke 	| 1 = patient previously had a stroke, 0 = has not had a stroke 	|
|       8      	|   prevalentHyp  	|     1 = patient was hypertensive, 0 = was not hypertensive    	|
|       9      	|     diabetes    	|      1 = patient had diabetes, 0 = did not have diabetes      	|
|      10      	|     totChol     	|                    Total cholesterol level (mg/dL)                    	|
|      11      	|      sysBP      	|                        Systolic blood pressure                        	|
|      12      	|      diaBP      	|                        Diastolic blood pressure                       	|
|      13      	|       BMI       	|                            Body mass index                            	|
|      14      	|    heartRate    	|                     Heart rate (BPM)                     	|
|      15      	|     glucose     	|                         Glucose level (mg/dL)                         	|
|      16      	|    TenYearCHD   	|   10 year risk of coronary heart disease, 1 = positive, 0 = negative  	|

In [26]:
# Percentage of positive CHD cases in the dataset.
sum(df['TenYearCHD'])/len(df)

0.1519584709768759

In [37]:
# Assessment of key attributes of the dataset.
def attributes(df):
    attrs = pd.DataFrame(columns=['N/A Count', 'Mean', 'Median', 'Min', 'Max'])
    for col in df:
        na_count = df[col].isna().sum()
        avg = round(df[col].mean(), 2)
        med = round(df[col].median(), 2)
        mn = min(df[col])
        mx = max(df[col])
        attrs.loc[col] = [na_count, avg, med, mn, mx]
    return attrs
    
    
attributes(df)
#sum(attributes(df)['N/A Count'])

,N/A Count,Mean,Median,Min,Max
male,0.0,0.43,0.0,0.00,1.0
age,0.0,49.58,49.0,32.00,70.0
education,105.0,1.98,2.0,1.00,4.0
currentSmoker,0.0,0.49,0.0,0.00,1.0
cigsPerDay,29.0,9.00,0.0,0.00,70.0
BPMeds,53.0,0.03,0.0,0.00,1.0
prevalentStroke,0.0,0.01,0.0,0.00,1.0
prevalentHyp,0.0,0.31,0.0,0.00,1.0
diabetes,0.0,0.03,0.0,0.00,1.0
totChol,50.0,236.72,234.0,107.00,696.0


There are quite a few incomplete entries that will need to be corrected. A mean attribute value will be used to substitute in for each missing value, with the intent that this will minimize the impact of these missing values on model training down the line. 
Attributes that consist of binary or categorical entries will be rounded to their nearest suitable value. 

In [28]:
# Replacement of NA values with attribute means. 
# Dataframe copies increase memory usage but allow for checkpoints.
df_na = df.copy()
size = len(df_na)

# Attributes that will have their NA values replaced by the attribute mean. 
na_cols_mean_replace = ['cigsPerDay', 'totChol', 'BMI', 'heartRate', 'glucose']

for col in na_cols_mean_replace:
    avg = round(df_na[col].mean(), 2)
    df_na[col] = df_na[col].fillna(avg)

# Education will be binned as is, so replacement values will need to correspond to an existing int. 
na_cols_mean_replace_round = ['education', 'BPMeds']

for col in na_cols_mean_replace_round:
    avg = round(df_na[col].mean(), 0)
    df_na[col] = df_na[col].fillna(avg)

# Verifying that no NA values remain.
attributes(df_na)

,N/A Count,Mean,Median,Min,Max
male,0.0,0.43,0.00,0.00,1.0
age,0.0,49.58,49.00,32.00,70.0
education,0.0,1.98,2.00,1.00,4.0
currentSmoker,0.0,0.49,0.00,0.00,1.0
cigsPerDay,0.0,9.00,0.00,0.00,70.0
BPMeds,0.0,0.03,0.00,0.00,1.0
prevalentStroke,0.0,0.01,0.00,0.00,1.0
prevalentHyp,0.0,0.31,0.00,0.00,1.0
diabetes,0.0,0.03,0.00,0.00,1.0
totChol,0.0,236.72,234.00,107.00,696.0


Next, attribute values will be binned into bins of at most 4 possible values. This turns continuous variables into categorical variables which are better suited for some of the applications down the line. 
Attribute values will be binned by quartiles. 

In [29]:
# Finds attributes that have more than 4 unique values. 
cols_to_bin = []
for col in df_na:
    if len(df_na[col].value_counts()) > 4:
        cols_to_bin.append(col)
        
print(cols_to_bin)

['age', 'cigsPerDay', 'totChol', 'sysBP', 'diaBP', 'BMI', 'heartRate', 'glucose']


In [30]:
# import time
# Speeds of various methods were calculated. Additional comment on this below. 

# In this cell the binning occurs, with the new attribute values based on quartile cutoffs. 

df_bin = df_na.copy()
size = len(df_bin)

for col in cols_to_bin:
    mn = round(min(df_bin[col]), 2)
    q1 = round(np.percentile(df_bin[col], 25), 2)
    q2 = round(df_bin[col].median(), 2)
    q3 = round(np.percentile(df_bin[col], 75), 2)
    mx = round(max(df_bin[col]), 2)
    
    to_q1 = '[' + str(mn) + ', ' + str(q1) + ']'
    to_q2 = '(' + str(q1) + ', ' + str(q2) + ']'
    to_q3 = '(' + str(q2) + ', ' + str(q3) + ']'
    to_q4 = '(' + str(q3) + ', ' + str(mx) + ']'

    # Using 'df_bin.loc[x, col]' is much slower than 'df_bin[col][x]' but prevents the 'SettingWithCopyWarning' error.
    for x in range(size):
        if df_bin.loc[x, col] <= q1:
            df_bin.loc[x, col] = to_q1
        elif df_bin.loc[x, col] <= q2:
            df_bin.loc[x, col] = to_q2
        elif df_bin.loc[x, col] <= q3:
            df_bin.loc[x, col] = to_q3
        else:
            df_bin.loc[x, col] = to_q4
    
df_bin.head(10)

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,"[32, 42.0]",4.0,0,"[0.0, 0.0]",0.0,0,0,0,"[107.0, 206.0]","[83.5, 117.0]","[48.0, 75.0]","(25.41, 28.04]","(75.0, 83.0]","(72.0, 80.0]",0
1,0,"(42.0, 49.0]",2.0,0,"[0.0, 0.0]",0.0,0,0,0,"(234.0, 262.0]","(117.0, 128.0]","(75.0, 82.0]","(28.04, 56.8]","(83.0, 143.0]","(72.0, 80.0]",0
2,1,"(42.0, 49.0]",1.0,1,"(0.0, 20.0]",0.0,0,0,0,"(234.0, 262.0]","(117.0, 128.0]","(75.0, 82.0]","(23.08, 25.41]","(68.0, 75.0]","[40.0, 72.0]",0
3,0,"(56.0, 70]",3.0,1,"(20.0, 70.0]",0.0,0,1,0,"(206.0, 234.0]","(144.0, 295.0]","(89.88, 142.5]","(28.04, 56.8]","[44.0, 68.0]","(85.0, 394.0]",1
4,0,"(42.0, 49.0]",3.0,1,"(20.0, 70.0]",0.0,0,0,0,"(262.0, 696.0]","(128.0, 144.0]","(82.0, 89.88]","(23.08, 25.41]","(83.0, 143.0]","(80.0, 85.0]",0
5,0,"(42.0, 49.0]",2.0,0,"[0.0, 0.0]",0.0,0,1,0,"(206.0, 234.0]","(144.0, 295.0]","(89.88, 142.5]","(28.04, 56.8]","(75.0, 83.0]","(85.0, 394.0]",0
6,0,"(56.0, 70]",1.0,0,"[0.0, 0.0]",0.0,0,0,0,"[107.0, 206.0]","(128.0, 144.0]","[48.0, 75.0]","(28.04, 56.8]","[44.0, 68.0]","(80.0, 85.0]",1
7,0,"(42.0, 49.0]",2.0,1,"(0.0, 20.0]",0.0,0,0,0,"(262.0, 696.0]","[83.5, 117.0]","[48.0, 75.0]","[15.54, 23.08]","(75.0, 83.0]","(72.0, 80.0]",0
8,1,"(49.0, 56.0]",1.0,0,"[0.0, 0.0]",0.0,0,1,0,"(234.0, 262.0]","(128.0, 144.0]","(82.0, 89.88]","(25.41, 28.04]","(75.0, 83.0]","(72.0, 80.0]",0
9,1,"(42.0, 49.0]",1.0,1,"(20.0, 70.0]",0.0,0,1,0,"(206.0, 234.0]","(144.0, 295.0]","(89.88, 142.5]","(23.08, 25.41]","(83.0, 143.0]","(85.0, 394.0]",0


Outcomes are converted from binary to 'yes'/'no' values. This will add human legibility to some steps down the line, as well as being inconsequential for any models that must operate using numeric values as all values in the dataset can be converted in one step. 

In [34]:
# Converts the binary outcome to 'yes'/'no' for human legibility. 
df_yn = df_bin.copy()
df_yn['TenYearCHD'] = np.where((df_yn['TenYearCHD'] == 1), 'yes', 'no')
df_yn.head(10)

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,"[32, 42.0]",4.0,0,"[0.0, 0.0]",0.0,0,0,0,"[107.0, 206.0]","[83.5, 117.0]","[48.0, 75.0]","(25.41, 28.04]","(75.0, 83.0]","(72.0, 80.0]",no
1,0,"(42.0, 49.0]",2.0,0,"[0.0, 0.0]",0.0,0,0,0,"(234.0, 262.0]","(117.0, 128.0]","(75.0, 82.0]","(28.04, 56.8]","(83.0, 143.0]","(72.0, 80.0]",no
2,1,"(42.0, 49.0]",1.0,1,"(0.0, 20.0]",0.0,0,0,0,"(234.0, 262.0]","(117.0, 128.0]","(75.0, 82.0]","(23.08, 25.41]","(68.0, 75.0]","[40.0, 72.0]",no
3,0,"(56.0, 70]",3.0,1,"(20.0, 70.0]",0.0,0,1,0,"(206.0, 234.0]","(144.0, 295.0]","(89.88, 142.5]","(28.04, 56.8]","[44.0, 68.0]","(85.0, 394.0]",yes
4,0,"(42.0, 49.0]",3.0,1,"(20.0, 70.0]",0.0,0,0,0,"(262.0, 696.0]","(128.0, 144.0]","(82.0, 89.88]","(23.08, 25.41]","(83.0, 143.0]","(80.0, 85.0]",no
5,0,"(42.0, 49.0]",2.0,0,"[0.0, 0.0]",0.0,0,1,0,"(206.0, 234.0]","(144.0, 295.0]","(89.88, 142.5]","(28.04, 56.8]","(75.0, 83.0]","(85.0, 394.0]",no
6,0,"(56.0, 70]",1.0,0,"[0.0, 0.0]",0.0,0,0,0,"[107.0, 206.0]","(128.0, 144.0]","[48.0, 75.0]","(28.04, 56.8]","[44.0, 68.0]","(80.0, 85.0]",yes
7,0,"(42.0, 49.0]",2.0,1,"(0.0, 20.0]",0.0,0,0,0,"(262.0, 696.0]","[83.5, 117.0]","[48.0, 75.0]","[15.54, 23.08]","(75.0, 83.0]","(72.0, 80.0]",no
8,1,"(49.0, 56.0]",1.0,0,"[0.0, 0.0]",0.0,0,1,0,"(234.0, 262.0]","(128.0, 144.0]","(82.0, 89.88]","(25.41, 28.04]","(75.0, 83.0]","(72.0, 80.0]",no
9,1,"(42.0, 49.0]",1.0,1,"(20.0, 70.0]",0.0,0,1,0,"(206.0, 234.0]","(144.0, 295.0]","(89.88, 142.5]","(23.08, 25.41]","(83.0, 143.0]","(85.0, 394.0]",no


Finally, the cleaned dataset is saved as a CSV file for models to be trained on. 

In [35]:
# Saves the cleaned dataframe to a CSV file for future analysis.  
df_yn.to_csv('framingham_cleaned.csv', index=False)